## Labeled Faces in the Wild

A benchmark dataset for Face Recognition

http://vis-www.cs.umass.edu/lfw/

...a database of face photographs designed for studying the problem of unconstrained face recognition. The data set contains more than 13,000 images of faces collected from the web. Each face has been labeled with the name of the person pictured. 1680 of the people pictured have two or more distinct photos in the data set. The only constraint on these faces is that they were detected by the Viola-Jones face detector.

In [1]:
from __future__ import print_function

from time import time
import logging
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people

%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# #############################################################################
# Download the data, if not already on disk and load it as numpy arrays

lfw_people = fetch_lfw_people(min_faces_per_person=10, color=True, slice_ = None)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w, c = lfw_people.images.shape

X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)
print("image size: %d, %d, %d" %(h,w,c))

Total dataset size:
n_samples: 4324
n_features: 46875
n_classes: 158
image size: 125, 125, 3


In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

num_classes = n_classes
# The data, split between train and test sets:
x_train, x_test, y_train, y_test = train_test_split(lfw_people.images, y, test_size=0.25, random_state=42, stratify=y)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('number of classes: ', num_classes)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



Using TensorFlow backend.


x_train shape: (3243, 125, 125, 3)
3243 train samples
1081 test samples
number of classes:  158


In [3]:
tb_callback = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,  
          write_graph=True, write_images=True)

batch_size = 50

epochs = 200
data_augmentation = True
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_lfw_trained_model.h5'

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,  callbacks=[tb_callback])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4, callbacks=[tb_callback])

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.


2018-05-07 09:32:06,574 Variable *= will be deprecated. Use variable.assign_mul if you want assignment to the variable value or 'x = x * y' if you want a new python Tensor object.


Epoch 1/200
65/65 [==============================] - 276s 4s/step - loss: 4.7063 - acc: 0.1123 - val_loss: 4.7128 - val_acc: 0.1295
Epoch 2/200
65/65 [==============================] - 273s 4s/step - loss: 4.6064 - acc: 0.1147 - val_loss: 4.6429 - val_acc: 0.1295
Epoch 3/200
65/65 [==============================] - 273s 4s/step - loss: 4.5386 - acc: 0.1173 - val_loss: 4.5524 - val_acc: 0.1341
Epoch 4/200
65/65 [==============================] - 273s 4s/step - loss: 4.4735 - acc: 0.1192 - val_loss: 4.4931 - val_acc: 0.1369
Epoch 5/200
65/65 [==============================] - 273s 4s/step - loss: 4.3938 - acc: 0.1215 - val_loss: 4.3503 - val_acc: 0.1378
Epoch 6/200
65/65 [==============================] - 273s 4s/step - loss: 4.3335 - acc: 0.1280 - val_loss: 4.2972 - val_acc: 0.1563
Epoch 7/200
65/65 [==============================] - 273s 4s/step - loss: 4.2690 - acc: 0.1307 - val_loss: 4.2044 - val_acc: 0.1656
Epoch 8/200
65/65 [==============================] - 273s 4s/step - loss: 4.